# This notebook preprocesses sample data related to sexual dimorphism research
## for reference, see 'Dimorph_samples.ods' file in /home/isaac/analysis/scRNA-seq
## # sample reader to read samples from /data/runs/samples
## and build each sample into a matrix

In [1]:
# Isaac Berez
# 14.01.23

from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods
from copy import deepcopy
import pprint
import json
import re
from datetime import datetime
import logging


import sample_reader as sr

### 1. get sample ids and construct meta data from ods file

In [2]:
samples,meta_data_dict = sr.process_meta_data(analysis_dir = '/home/isaac/analysis/scRNA-seq/',
                                dimorph_sample_file = 'Dimorph_samples.ods')
pprint.pprint(meta_data_dict)

{'10X35_1': {'Age': '8w',
             'Avesizebp_cDNAlib': nan,
             'Cell_Conc': 1000.0,
             'ChipID': '10X35',
             'Comments': 'Naive 8wo females, pooled, left hemisphere. 5%opti.',
             'Date': None,
             'Date_Captured': '2019-11-20',
             'DonorID': 'DI1,DI2',
             'Group': 'Naïve-F',
             'LIbConstructionComment': None,
             'Num_Pooled_Animals': 2.0,
             'PCR_Cycles': 13.0,
             'Project': 'Dimorph',
             'SampleID': '10X35_1',
             'Sample_Index': 'D9',
             'Serial_Number': 63.0,
             'Sex': 'F',
             'Species': 'Mm',
             'Strain': None,
             'Target_Num_Cells': 5000.0,
             'Tissue': 'MeA_L',
             'Transcriptome': 'Mm10',
             'cDNA_Lib_Ok': 'Y',
             'cDNAul': nan,
             'lengthbp_seqlib': 416.0,
             'ngperul_cDNA': 23.3,
             'ngperul_seqlib': 13.6},
 '10X35_2': {'Age': '8

### 2. Validatation of find latest file function

In [3]:
#testing find latest sample file on all dimorph sample files
sample_dir = '/data/runs/samples'
count = 0
for i in range(len(samples)):
    file = sr.find_latest_sample_file(sample_dir,samples,i)
    print ('Passed!')
    count+=1
print (f'expected {len(samples)} sample files, succesfully found {count} sample files')

finding latest sample files from:  ['commands_10X54_1_200707', 'out10X54_1_200707']
found latest file:  out10X54_1_200707
Passed!
finding latest sample files from:  ['commands_10X54_2_210220', 'out10X54_2_210220', 'commands_10X54_2_200707', 'out10X54_2_200707']
found latest file:  out10X54_2_210220
Passed!
finding latest sample files from:  ['commands_10X98_2_221010', 'out10X98_2_221010']
found latest file:  out10X98_2_221010
Passed!
finding latest sample files from:  ['commands_10X98_3_221010', 'out10X98_3_221010']
found latest file:  out10X98_3_221010
Passed!
finding latest sample files from:  ['out10X51_2_210220', 'commands_10X51_2_200608', 'out10X51_2_200608', 'out10X51_2_200707', 'commands_10X51_2_200707', '10xMtxFile.mat', 'commands_10X51_2_210220']
found latest file:  out10X51_2_210220
Passed!
finding latest sample files from:  ['commands_10X51_1_200707', 'commands_10X51_1_200608', 'out10X51_1_210220', 'commands_10X51_1_210220', '10xMtxFile.mat', 'out10X51_1_200608', 'out10X51_1

### 3. Test construct_sample matrix on some samples, samples found in '/data/runs/samples/'

In [4]:
df1 = sr.construct_sample_matrix(sample_dir= '/data/runs/samples/',samples=samples,sample_ind=0)
#construct_sample_matrix(sample_dir= '/data/runs/samples/',samples=samples,sample_ind=0)
df1

finding latest sample files from:  ['commands_10X54_1_200707', 'out10X54_1_200707']
found latest file:  out10X54_1_200707
sample df after adding/removing duplicate rows:  (27933, 6229)


,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)","(AAACGAATCTGCTTAT-110X54_1,)","(AAACGCTAGCAGATAT-110X54_1,)","(AAACGCTAGTGCTCAT-110X54_1,)","(AAACGCTGTCCGTACG-110X54_1,)","(AAACGCTGTGGCTACC-110X54_1,)",...,"(TTTGGTTAGAGTCACG-110X54_1,)","(TTTGGTTCACAGCCTG-110X54_1,)","(TTTGGTTCAGCCTTCT-110X54_1,)","(TTTGGTTGTCACTAGT-110X54_1,)","(TTTGGTTGTTCCTAAG-110X54_1,)","(TTTGGTTTCATCTGTT-110X54_1,)","(TTTGGTTTCTGTGCTC-110X54_1,)","(TTTGTTGAGATGTTCC-110X54_1,)","(TTTGTTGCATAGCACT-110X54_1,)","(TTTGTTGTCCGCGGAT-110X54_1,)"
"(0610007P14Rik,)",0,1,5,0,0,0,1,0,2,1,...,1,2,0,0,1,0,0,0,0,0
"(0610009B22Rik,)",0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
"(0610009L18Rik,)",0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(0610009O20Rik,)",0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
"(0610010F05Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",1,9,17,21,9,6,14,23,10,10,...,16,10,4,1,3,4,4,8,14,1
"(mt-Nd4,)",10,60,117,66,30,50,87,144,80,53,...,82,76,44,12,9,38,10,57,73,24
"(mt-Nd4l,)",0,7,8,3,4,3,1,5,10,2,...,8,8,2,0,0,2,2,2,6,1
"(mt-Nd5,)",3,7,14,8,5,5,7,19,9,5,...,14,10,0,0,1,7,0,6,11,3


### 4. Run on a few other samples

In [5]:
df2 = sr.construct_sample_matrix(sample_dir= '/data/runs/samples/',samples=samples,sample_ind=1)
df2

finding latest sample files from:  ['commands_10X54_2_210220', 'out10X54_2_210220', 'commands_10X54_2_200707', 'out10X54_2_200707']
found latest file:  out10X54_2_210220
sample df after adding/removing duplicate rows:  (27933, 4343)


,"(AAACCCACATGTTACG-110X54_2,)","(AAACGAAAGACGAAGA-110X54_2,)","(AAACGAAGTTCTCAGA-110X54_2,)","(AAACGAATCAGCTAGT-110X54_2,)","(AAACGAATCTCTAAGG-110X54_2,)","(AAACGAATCTCTGAGA-110X54_2,)","(AAACGCTAGAATCTAG-110X54_2,)","(AAACGCTAGTCTGCGC-110X54_2,)","(AAACGCTTCTCTGACC-110X54_2,)","(AAACGCTTCTGGTTGA-110X54_2,)",...,"(TTTGGAGAGTGGTTCT-110X54_2,)","(TTTGGAGCATGGCCCA-110X54_2,)","(TTTGGAGTCACAAGAA-110X54_2,)","(TTTGGAGTCCATAGGT-110X54_2,)","(TTTGGAGTCTCTCGAC-110X54_2,)","(TTTGGTTAGAGTGGCT-110X54_2,)","(TTTGGTTAGCTACTAC-110X54_2,)","(TTTGGTTCAGCGCTTG-110X54_2,)","(TTTGGTTCATGACAGG-110X54_2,)","(TTTGTTGTCTACTGCC-110X54_2,)"
"(0610007P14Rik,)",0,0,0,1,0,0,0,2,0,0,...,0,0,2,0,2,2,1,0,2,1
"(0610009B22Rik,)",1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
"(0610009L18Rik,)",1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
"(0610009O20Rik,)",1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
"(0610010F05Rik,)",1,0,0,0,1,1,0,0,0,1,...,0,0,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",28,10,8,18,2,19,2,19,32,3,...,5,0,9,6,3,7,26,22,0,16
"(mt-Nd4,)",69,57,56,81,4,68,5,53,179,26,...,22,13,58,27,42,24,132,119,58,48
"(mt-Nd4l,)",10,6,5,9,2,3,2,3,10,2,...,0,0,5,1,2,1,12,9,5,2
"(mt-Nd5,)",13,8,5,21,1,8,2,2,15,3,...,1,1,7,5,5,1,12,11,6,11


In [7]:
df3 = sr.construct_sample_matrix(sample_dir= '/data/runs/samples/',samples=samples,sample_ind=2)
df3

finding latest sample files from:  ['commands_10X98_2_221010', 'out10X98_2_221010']
found latest file:  out10X98_2_221010
sample df after adding/removing duplicate rows:  (27933, 4725)


,"(AAACCCACAATAGTCC-110X98_2,)","(AAACGAAAGGAGATAG-110X98_2,)","(AAACGAAAGGTACATA-110X98_2,)","(AAACGAACAAACAGGC-110X98_2,)","(AAACGAACAAATCGTC-110X98_2,)","(AAACGAACAACACGAG-110X98_2,)","(AAACGAAGTTGGAGAC-110X98_2,)","(AAACGAATCCCGAACG-110X98_2,)","(AAACGAATCCGATGCG-110X98_2,)","(AAACGAATCTTGTTAC-110X98_2,)",...,"(TTTGATCGTCCGAAAG-110X98_2,)","(TTTGGAGAGGTGCTGA-110X98_2,)","(TTTGGAGCAGTCTTCC-110X98_2,)","(TTTGGAGCATCGAACT-110X98_2,)","(TTTGGAGGTCGCATTA-110X98_2,)","(TTTGGTTCATTAGGCT-110X98_2,)","(TTTGGTTGTTCCCACT-110X98_2,)","(TTTGGTTTCCGATAAC-110X98_2,)","(TTTGTTGTCTCTAAGG-110X98_2,)","(TTTGTTGTCTGAGAGG-110X98_2,)"
"(0610007P14Rik,)",1,0,2,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"(0610009B22Rik,)",0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
"(0610009L18Rik,)",0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
"(0610009O20Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
"(0610010F05Rik,)",0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",4,1,3,8,8,4,2,3,6,6,...,0,0,3,5,2,0,2,11,3,2
"(mt-Nd4,)",9,3,24,54,30,25,5,10,30,13,...,4,0,33,39,8,5,8,32,27,18
"(mt-Nd4l,)",0,0,1,6,1,1,0,1,3,1,...,0,0,1,7,0,0,2,1,2,0
"(mt-Nd5,)",1,1,2,9,4,5,0,0,9,2,...,1,0,3,5,0,2,3,7,3,3


### 5. Check that rows (gene labels) match

In [8]:
(np.array(df1.index)==np.array(df2.index)).all()

True

### 6. Calculate expected shape when combining df1 and df2 and verify join method

In [13]:
print (f'df1 shape {df1.shape}')
print (f'df2 shape {df2.shape}')
combined_col_length = df1.shape[1]+df2.shape[1]
print (f'expected shape of combined df1 and df2 {df1.shape[0]},{combined_col_length}')

df1 shape (27933, 6229)
df2 shape (27933, 4343)
expected shape of combined df1 and df2 27933,10572


In [9]:
#df1.merge(df2, left_index=True, right_index=True)

In [46]:
dfc = pd.DataFrame(index=df1.index)
dfc

""
"(0610007P14Rik,)"
"(0610009B22Rik,)"
"(0610009L18Rik,)"
"(0610009O20Rik,)"
"(0610010F05Rik,)"
...
"(mt-Nd3,)"
"(mt-Nd4,)"
"(mt-Nd4l,)"
"(mt-Nd5,)"


#### test joining of df1 and df2

In [14]:
#sample_dir = '/data/runs/samples'
dfc = pd.DataFrame(index=df1.index)
for i in range(2):
    df = sr.construct_sample_matrix(sample_dir,samples,i)
    dfc = dfc.join(df)
print(f'combined df1 and df2 shape: {dfc.shape}, expected: {df1.shape[0]},{combined_col_length}')

finding latest sample files from:  ['commands_10X54_1_200707', 'out10X54_1_200707']
found latest file:  out10X54_1_200707
sample df after adding/removing duplicate rows:  (27933, 6229)
finding latest sample files from:  ['commands_10X54_2_210220', 'out10X54_2_210220', 'commands_10X54_2_200707', 'out10X54_2_200707']
found latest file:  out10X54_2_210220
sample df after adding/removing duplicate rows:  (27933, 4343)
combined df1 and df2 shape: (27933, 10572), expected: 27933,10572


In [52]:
dfc

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)","(AAACGAATCTGCTTAT-110X54_1,)","(AAACGCTAGCAGATAT-110X54_1,)","(AAACGCTAGTGCTCAT-110X54_1,)","(AAACGCTGTCCGTACG-110X54_1,)","(AAACGCTGTGGCTACC-110X54_1,)",...,"(TTTGGAGAGTGGTTCT-110X54_2,)","(TTTGGAGCATGGCCCA-110X54_2,)","(TTTGGAGTCACAAGAA-110X54_2,)","(TTTGGAGTCCATAGGT-110X54_2,)","(TTTGGAGTCTCTCGAC-110X54_2,)","(TTTGGTTAGAGTGGCT-110X54_2,)","(TTTGGTTAGCTACTAC-110X54_2,)","(TTTGGTTCAGCGCTTG-110X54_2,)","(TTTGGTTCATGACAGG-110X54_2,)","(TTTGTTGTCTACTGCC-110X54_2,)"
"(0610007P14Rik,)",0,1,5,0,0,0,1,0,2,1,...,0,0,2,0,2,2,1,0,2,1
"(0610009B22Rik,)",0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
"(0610009L18Rik,)",0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
"(0610009O20Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
"(0610010F05Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",1,9,17,21,9,6,14,23,10,10,...,5,0,9,6,3,7,26,22,0,16
"(mt-Nd4,)",10,60,117,66,30,50,87,144,80,53,...,22,13,58,27,42,24,132,119,58,48
"(mt-Nd4l,)",0,7,8,3,4,3,1,5,10,2,...,0,0,5,1,2,1,12,9,5,2
"(mt-Nd5,)",3,7,14,8,5,5,7,19,9,5,...,1,1,7,5,5,1,12,11,6,11


### 7. Combine all samples into single dataframe

In [55]:
df_all = pd.DataFrame(index=df1.index)
for i in range(len(samples)):
    df = sr.construct_sample_matrix(sample_dir,samples,i)
    df_all = df_all.join(df)

finding latest sample files from:  ['commands_10X54_1_200707', 'out10X54_1_200707']
found latest file:  out10X54_1_200707
sample df after adding/removing duplicate rows:  (27933, 6229)
finding latest sample files from:  ['commands_10X54_2_210220', 'out10X54_2_210220', 'commands_10X54_2_200707', 'out10X54_2_200707']
found latest file:  out10X54_2_210220
sample df after adding/removing duplicate rows:  (27933, 4343)
finding latest sample files from:  ['commands_10X98_2_221010', 'out10X98_2_221010']
found latest file:  out10X98_2_221010
sample df after adding/removing duplicate rows:  (27933, 4725)
finding latest sample files from:  ['commands_10X98_3_221010', 'out10X98_3_221010']
found latest file:  out10X98_3_221010
sample df after adding/removing duplicate rows:  (27933, 6407)
finding latest sample files from:  ['out10X51_2_210220', 'commands_10X51_2_200608', 'out10X51_2_200608', 'out10X51_2_200707', 'commands_10X51_2_200707', '10xMtxFile.mat', 'commands_10X51_2_210220']
found latest f

In [56]:
df_all

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)","(AAACGAATCTGCTTAT-110X54_1,)","(AAACGCTAGCAGATAT-110X54_1,)","(AAACGCTAGTGCTCAT-110X54_1,)","(AAACGCTGTCCGTACG-110X54_1,)","(AAACGCTGTGGCTACC-110X54_1,)",...,"(TTTGGTTTCCACGAAT-110X37_2,)","(TTTGGTTTCTTTCTAG-110X37_2,)","(TTTGTTGAGAAATTCG-110X37_2,)","(TTTGTTGAGTGCTCGC-110X37_2,)","(TTTGTTGCAAGGTCAG-110X37_2,)","(TTTGTTGCACCGCTGA-110X37_2,)","(TTTGTTGGTCCATAGT-110X37_2,)","(TTTGTTGGTTGCGAAG-110X37_2,)","(TTTGTTGTCGAAGGAC-110X37_2,)","(TTTGTTGTCTCTGCTG-110X37_2,)"
"(0610007P14Rik,)",0,1,5,0,0,0,1,0,2,1,...,1,0,1,0,2,1,1,0,0,0
"(0610009B22Rik,)",0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
"(0610009L18Rik,)",0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
"(0610009O20Rik,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(0610010F05Rik,)",0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(mt-Nd3,)",1,9,17,21,9,6,14,23,10,10,...,10,6,34,15,33,12,2,4,4,1
"(mt-Nd4,)",10,60,117,66,30,50,87,144,80,53,...,90,40,170,69,130,67,33,31,38,31
"(mt-Nd4l,)",0,7,8,3,4,3,1,5,10,2,...,14,4,13,4,8,5,9,7,3,5
"(mt-Nd5,)",3,7,14,8,5,5,7,19,9,5,...,17,5,24,4,18,13,6,2,4,6


In [57]:
df_all.index

Index([('0610007P14Rik',), ('0610009B22Rik',), ('0610009L18Rik',),
       ('0610009O20Rik',), ('0610010F05Rik',), ('0610010K14Rik',),
       ('0610011F06Rik',), ('0610012D04Rik',), ('0610012G03Rik',),
       ('0610025J13Rik',),
       ...
              ('mt-Co2',),        ('mt-Co3',),       ('mt-Cytb',),
              ('mt-Nd1',),        ('mt-Nd2',),        ('mt-Nd3',),
              ('mt-Nd4',),       ('mt-Nd4l',),        ('mt-Nd5',),
              ('mt-Nd6',)],
      dtype='object', length=27933)

### 8. Remove trailing comma in all rows and columns

In [58]:
genes = [x[0] for x in df_all.index]
cells = [x[0] for x in df_all.columns]

In [59]:
df_all_copy = df_all.copy()
df_all_copy.index = genes
df_all_copy.columns = cells
df_all_copy

,AAACCCACAACAGTGG-110X54_1,AAACCCACATGGCCCA-110X54_1,AAACCCAGTCCCTGAG-110X54_1,AAACGAACACTACAGT-110X54_1,AAACGAATCCCAGCGA-110X54_1,AAACGAATCTGCTTAT-110X54_1,AAACGCTAGCAGATAT-110X54_1,AAACGCTAGTGCTCAT-110X54_1,AAACGCTGTCCGTACG-110X54_1,AAACGCTGTGGCTACC-110X54_1,...,TTTGGTTTCCACGAAT-110X37_2,TTTGGTTTCTTTCTAG-110X37_2,TTTGTTGAGAAATTCG-110X37_2,TTTGTTGAGTGCTCGC-110X37_2,TTTGTTGCAAGGTCAG-110X37_2,TTTGTTGCACCGCTGA-110X37_2,TTTGTTGGTCCATAGT-110X37_2,TTTGTTGGTTGCGAAG-110X37_2,TTTGTTGTCGAAGGAC-110X37_2,TTTGTTGTCTCTGCTG-110X37_2
0610007P14Rik,0,1,5,0,0,0,1,0,2,1,...,1,0,1,0,2,1,1,0,0,0
0610009B22Rik,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
0610009L18Rik,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
0610009O20Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010F05Rik,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,1,9,17,21,9,6,14,23,10,10,...,10,6,34,15,33,12,2,4,4,1
mt-Nd4,10,60,117,66,30,50,87,144,80,53,...,90,40,170,69,130,67,33,31,38,31
mt-Nd4l,0,7,8,3,4,3,1,5,10,2,...,14,4,13,4,8,5,9,7,3,5
mt-Nd5,3,7,14,8,5,5,7,19,9,5,...,17,5,24,4,18,13,6,2,4,6


### 9. Write completed data matrix to file

In [ ]:
df_all_copy.to_csv('/bigdata/isaac/dimorph_df.csv', index = True, index_label=False)

In [66]:
test_df = df_all.iloc[:5,:5].copy()

In [68]:
test_df

,"(AAACCCACAACAGTGG-110X54_1,)","(AAACCCACATGGCCCA-110X54_1,)","(AAACCCAGTCCCTGAG-110X54_1,)","(AAACGAACACTACAGT-110X54_1,)","(AAACGAATCCCAGCGA-110X54_1,)"
"(0610007P14Rik,)",0,1,5,0,0
"(0610009B22Rik,)",0,0,0,1,0
"(0610009L18Rik,)",0,0,0,0,1
"(0610009O20Rik,)",0,0,0,0,0
"(0610010F05Rik,)",0,0,0,0,0


In [70]:
test_df.to_csv('/bigdata/isaac/test_df.csv', index = True, index_label=False)